In [1]:
# Use modules from parent directory
import os
import sys
import tifffile as tiff
import numpy as np
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000   
from matplotlib import pyplot as plt

sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2
from srcv2_2.models.params import get_params
from srcv2_2.models.Unet import Unet, UnetV2
from srcv2_2.visualization.visualization_utils import get_predicted_thumbnails 
from srcv2_2.utils import get_model_name

import tensorflow as tf

2024-04-10 16:33:43.047621: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-10 16:33:43.065428: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 16:33:43.065447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 16:33:43.065833: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-10 16:33:43.068733: I tensorflow/core/platform/cpu_feature_guar

## Defining Params

For development purposes, use epochs=5. For final use, use epochs=20 perhaps.

In [2]:
params = get_params('U-net', 'MODIS')
params.cls = ['cloud', 'thin', 'clear', 'shadow'] #'clear', 'cloud', 'shadow', 'snow', 'water'] 

#working edition: ['cloud', 'thin']
params.collapse_cls = False
params.project_path = "/home/mxh/RS-Net/"
params.activation_func='relu'
params.last_layer_activation_func = 'softmax'
params.loss_func="categorical_crossentropy"
params.learning_rate = 1e-4
params.epochs = 5
params.threshold = 0.5
params.dropout = 0.2 # 0
params.L2reg = 1e-4 # 0
params.decay = 0.0
params.num_gpus = 1
params.overlap = 40
params.modelID = "_v2_overlap40_mc_cloud-thin-clear-shadow_Last-Layer-softmax_v5"
params.norm_threshold = 3500

#tile = 'all'  # Name of one tile or 'all'
#tile = 'L2A_T32UNF_20170803T103019' 

## Training of local model

In [3]:
params.__dict__

{'modelNick': 'Unet-MOD09GA',
 'modelID': '_v2_overlap40_mc_cloud-thin-clear-shadow_Last-Layer-softmax_v5',
 'num_gpus': 1,
 'optimizer': 'Adam',
 'loss_func': 'categorical_crossentropy',
 'activation_func': 'relu',
 'last_layer_activation_func': 'softmax',
 'initialization': 'glorot_uniform',
 'use_batch_norm': True,
 'dropout_on_last_layer_only': True,
 'early_stopping': False,
 'reduce_lr': False,
 'save_best_only': False,
 'use_ensemble_learning': False,
 'ensemble_method': 'Bagging',
 'learning_rate': 0.0001,
 'dropout': 0.2,
 'L1reg': 0.0,
 'L2reg': 0.0001,
 'L1L2reg': 0.0,
 'decay': 0.0,
 'batch_norm_momentum': 0.7,
 'threshold': 0.5,
 'patch_size': 256,
 'overlap': 40,
 'overlap_train_set': 0,
 'batch_size': 40,
 'steps_per_epoch': None,
 'epochs': 5,
 'norm_method': 'enhance_contrast',
 'norm_threshold': 3500,
 'cls': ['cloud', 'thin', 'clear', 'shadow'],
 'collapse_cls': False,
 'affine_transformation': True,
 'brightness_augmentation': False,
 'bands': [1, 2, 3, 4, 5, 6, 7],

In [4]:
# training local (notebook) model
local_model = UnetV2(params)
local_model.get_config()

2024-04-10 16:33:49.680990: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 16:33:49.702732: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 16:33:49.702846: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

No weights found


{'seed': 1,
 'params': <srcv2_2.models.params.HParams at 0x7fbf6df1e3d0>,
 'n_cls': 4,
 'n_bands': 7,
 'model_config': {'name': 'model',
  'trainable': True,
  'layers': [{'module': 'keras.layers',
    'class_name': 'InputLayer',
    'config': {'batch_input_shape': (None, 256, 256, 7),
     'dtype': 'float32',
     'sparse': False,
     'ragged': False,
     'name': 'input_1'},
    'registered_name': None,
    'name': 'input_1',
    'inbound_nodes': []},
   {'module': 'keras.layers',
    'class_name': 'Conv2D',
    'config': {'name': 'conv2d',
     'trainable': True,
     'dtype': 'float32',
     'filters': 32,
     'kernel_size': (3, 3),
     'strides': (1, 1),
     'padding': 'same',
     'data_format': 'channels_last',
     'dilation_rate': (1, 1),
     'groups': 1,
     'activation': 'relu',
     'use_bias': True,
     'kernel_initializer': {'module': 'keras.initializers',
      'class_name': 'GlorotUniform',
      'config': {'seed': None},
      'registered_name': None},
     'bia

In [5]:
local_model.train()

Compiling with Categorical Crossentropy
------------------------------------------
Start training:
Epoch 1/5


2024-04-10 16:33:52.517132: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-04-10 16:33:53.145714: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8800


477/477 [==============================] - 141s 262ms/step - loss: 1.5922 - categorical_accuracy: 0.2288 - categorical_crossentropy: 0.0000e+00 - jaccard_coef_loss: 40.5512 - jaccard_coef: 1.1574e-16 - jaccard_coef_thresholded: 1.4079e-16 - accuracy: 0.7152 - val_loss: 5.0197 - val_categorical_accuracy: 0.0000e+00 - val_categorical_crossentropy: 0.0000e+00 - val_jaccard_coef_loss: 40.5514 - val_jaccard_coef: 1.1574e-16 - val_jaccard_coef_thresholded: 1.1574e-16 - val_accuracy: 0.7500
Epoch 2/5
477/477 [==============================] - 135s 282ms/step - loss: 13.1872 - categorical_accuracy: 0.1186 - categorical_crossentropy: 0.0000e+00 - jaccard_coef_loss: 40.5512 - jaccard_coef: 1.1574e-16 - jaccard_coef_thresholded: 1.1574e-16 - accuracy: 0.7500 - val_loss: 23.5640 - val_categorical_accuracy: 0.0000e+00 - val_categorical_crossentropy: 0.0000e+00 - val_jaccard_coef_loss: 40.5514 - val_jaccard_coef: 1.1574e-16 - val_jaccard_coef_thresholded: 1.1574e-16 - val_accuracy: 0.7500
Epoch 3/5


In [6]:
get_model_name(params)

'Unet_MODIS__v2_overlap40_mc_cloud-thin-clear-shadow_Last-Layer-softmax_v2'

In [8]:
# check if training_params are saved
local_model.get_config()["params"].__dict__

{'modelNick': 'Unet-MOD09GA',
 'modelID': '_v2_overlap40_mc_cloud-thin-clear-shadow',
 'num_gpus': 1,
 'optimizer': 'Adam',
 'loss_func': 'binary_crossentropy',
 'activation_func': 'softmax',
 'initialization': 'glorot_uniform',
 'use_batch_norm': True,
 'dropout_on_last_layer_only': True,
 'early_stopping': False,
 'reduce_lr': False,
 'save_best_only': False,
 'use_ensemble_learning': False,
 'ensemble_method': 'Bagging',
 'learning_rate': 0.0001,
 'dropout': 0,
 'L1reg': 0.0,
 'L2reg': 0,
 'L1L2reg': 0.0,
 'decay': 0,
 'batch_norm_momentum': 0.7,
 'threshold': 0.5,
 'patch_size': 256,
 'overlap': 40,
 'overlap_train_set': 0,
 'batch_size': 40,
 'steps_per_epoch': None,
 'epochs': 10,
 'norm_method': 'enhance_contrast',
 'norm_threshold': 3500,
 'cls': ['cloud', 'thin', 'clear', 'shadow'],
 'collapse_cls': False,
 'affine_transformation': True,
 'brightness_augmentation': False,
 'bands': [1, 2, 3, 4, 5, 6, 7],
 'project_path': '/home/mxh/RS-Net/',
 'toa_path': 'data/processed/Biome_